<a href="https://colab.research.google.com/github/devnen/Chatterbox-TTS-Server/blob/main/Chatterbox_TTS_Server_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. Setup and Installation (Robust Version)
# Clone repository and install dependencies with error handling

# Remove existing directory and clear cache
!rm -rf Chatterbox-TTS-Server
!pip cache purge

# Clone repository
!git clone https://github.com/devnen/Chatterbox-TTS-Server.git
%cd Chatterbox-TTS-Server

print("✅ Repository cloned. Installing core dependencies...")

# Install PyTorch with compatible torchvision
!pip install torch==2.5.1+cu121 torchaudio==2.5.1+cu121 torchvision==0.20.1+cu121 --index-url https://download.pytorch.org/whl/cu121 -q

# Install your Colab-compatible chatterbox fork
!pip install git+https://github.com/devnen/chatterbox.git -q

print("✅ Core TTS components installed. Installing server dependencies...")

# Install essential server requirements (skip problematic packages)
!pip install fastapi uvicorn pyyaml soundfile librosa safetensors -q
!pip install python-multipart requests jinja2 watchdog aiofiles unidecode inflect tqdm -q
!pip install pydub audiotsm -q

# Try to install parselmouth (may fail on some systems)
!pip install parselmouth -q || echo "Parselmouth installation failed - unvoiced segment removal will be disabled"

print("✅ Installation complete! Some optional packages may have been skipped.")

Files removed: 40
Cloning into 'Chatterbox-TTS-Server'...
remote: Enumerating objects: 193, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 193 (delta 37), reused 29 (delta 24), pack-reused 139 (from 1)
Receiving objects: 100% (193/193), 18.89 MiB | 16.51 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/Chatterbox-TTS-Server
✅ Repository cloned. Installing core dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
✅ Core TTS components installed. Installing server dependencies...
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for outpu

In [1]:
# @title 2. Launch the Server! ⚡
# Starts the TTS server with memory optimization

import threading
import time
import uvicorn
import gc
from google.colab.output import serve_kernel_port_as_window

port = 8004

print("🛑 Stopping existing servers...")
!fuser -k 8004/tcp 2>/dev/null || echo "Port clear"
!pkill -f "uvicorn.*server" 2>/dev/null || echo "No running servers"

print("🧹 Clearing memory...")
gc.collect()

%cd /content/Chatterbox-TTS-Server

def run_server():
    try:
        from server import app
        print("✅ Server starting...")
        # Add memory-efficient settings
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=port,
            log_level="warning",  # Reduce log verbosity
            access_log=False      # Disable access logs to save memory
        )
    except Exception as e:
        print(f"❌ Server error: {e}")

print("🚀 Starting TTS server...")

thread = threading.Thread(target=run_server, daemon=True)
thread.start()

time.sleep(25)  # Extra time for model loading

print("🎉 Server ready! Click below:")
serve_kernel_port_as_window(port)

print("\n💡 Tips to prevent crashes:")
print("• Use shorter texts (under 500 characters)")
print("• Wait for each generation to complete before starting another")
print("• If it crashes, just re-run this cell")

🛑 Stopping existing servers...
Port clear
^C
🧹 Clearing memory...
/content/Chatterbox-TTS-Server
🚀 Starting TTS server...


✅ Server starting...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/diffusers/models/lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)


🎉 Server ready! Click below:
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>


💡 Tips to prevent crashes:
• Use shorter texts (under 500 characters)
• Wait for each generation to complete before starting another
• If it crashes, just re-run this cell


In [ ]:
# @title 3. Server Health Monitor (Optional)
# Run this if you experience crashes

import psutil
import time

def monitor_resources():
    for i in range(10):
        # Check memory usage
        memory = psutil.virtual_memory()
        print(f"RAM: {memory.percent:.1f}% used | Available: {memory.available//1024//1024//1024:.1f}GB")

        # Check if server is still running
        server_running = any("server" in p.name() for p in psutil.process_iter(['name']))
        print(f"Server status: {'🟢 Running' if server_running else '🔴 Stopped'}")

        time.sleep(10)
        print("---")

print("🔍 Monitoring server resources...")
monitor_resources()
